# Flight Delay Predictions with Weather Company Data, FlightStats, and Apache Spark MLlib  

<img style="max-width: 800px; padding: 25px 0px;" src="https://ibm-cds-labs.github.io/simple-data-pipe-connector-flightstats/flight_predictor_architecture.png"/>
  
The following notebook features a Spark Machine Learning application that predicts whether a flight will be delayed based on weather data.  

The application workflow is as follow:  
1. Configure the application parameters
2. Load the training and test data
3. Build the classification models
4. Evaluate the models and iterate
5. Launch a Pixiedust embedded application to run the models  

## Pre-requisite  

This notebook is a follow up to [Predict Flight Delays with Apache Spark MLlib, FlightStats, and Weather Data](https://developer.ibm.com/clouddataservices/2016/08/04/predict-flight-delays-with-apache-spark-mllib-flightstats-and-weather-data/). Please follow the instructions in the article and:

* Set up a FlightStats account  
* Provision the Weather Company Data service  
* Obtain or build the training and test data sets  

## Learn more  

* [Weather Company Data](https://console.ng.bluemix.net/docs/services/Weather/index.html)  
* [FlightStats](https://developer.flightstats.com/)  
* [Apache Spark MLlib](https://spark.apache.org/mllib/)  
* [PixieDust](https://github.com/ibm-cds-labs/pixiedust)  
* [pixiedust_flightpredict](https://github.com/ibm-cds-labs/simple-data-pipe-connector-flightstats/tree/master/pixiedust_flightpredict)    

# Install latest pixiedust  and pixiedust-flightpredict plugin

Make sure you are running the latest `pixiedust` and `pixiedust-flightpredict` versions. After upgrading restart the kernel before continuing to the next cells.

In [ ]:
!pip install --upgrade --user pixiedust

In [ ]:
!pip install --upgrade --user pixiedust-flightpredict

<h3>If PixieDust was just installed or upgraded, <span style="color: red">restart the kernel</span> before continuing.</h3>

# Import required python package and set the Cloudant credentials  

Have available your credentials for Cloudant, Weather Company Data, and FlightStats, as well as the training and test data info from [Predict Flight Delays with Apache Spark MLlib, FlightStats, and Weather Data](https://developer.ibm.com/clouddataservices/2016/08/04/predict-flight-delays-with-apache-spark-mllib-flightstats-and-weather-data/)  

Run through the Configuration Dashboard to load the training and test data. Ensure all <i class="fa fa-2x fa-times" style="font-size:medium"></i> statuses are completed. After applying changes, re-run the below cell to confirm the updated configuration statuses.

In [ ]:
import pixiedust_flightpredict
pixiedust_flightpredict.configure()

# Train multiple classification models  

The below cells train four models: Logistic Regression, Naive Bayes, Decision Tree, and Random Forest.
Feel free to update these models or build your own models.

In [ ]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors
from numpy import array
import numpy as np
import math
from datetime import datetime
from dateutil import parser
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
logRegModel = LogisticRegressionWithLBFGS.train(labeledTrainingData.map(lambda lp: LabeledPoint(lp.label,\
      np.fromiter(map(lambda x: 0.0 if np.isnan(x) else x,lp.features.toArray()),dtype=np.double )))\
      , iterations=1000, validateData=False, intercept=False)
print(logRegModel)

In [ ]:
from pyspark.mllib.classification import NaiveBayes
#NaiveBayes requires non negative features, set them to 0 for now
modelNaiveBayes = NaiveBayes.train(labeledTrainingData.map(lambda lp: LabeledPoint(lp.label, \
                    np.fromiter(map(lambda x: x if x>0.0 else 0.0,lp.features.toArray()),dtype=np.int)\
               ))\
          )

print(modelNaiveBayes)

In [ ]:
from pyspark.mllib.tree import DecisionTree
modelDecisionTree = DecisionTree.trainClassifier(labeledTrainingData.map(lambda lp: LabeledPoint(lp.label,\
      np.fromiter(map(lambda x: 0.0 if np.isnan(x) else x,lp.features.toArray()),dtype=np.double )))\
      , numClasses=training.getNumClasses(), categoricalFeaturesInfo={})
print(modelDecisionTree)

In [ ]:
from pyspark.mllib.tree import RandomForest
modelRandomForest = RandomForest.trainClassifier(labeledTrainingData.map(lambda lp: LabeledPoint(lp.label,\
      np.fromiter(map(lambda x: 0.0 if np.isnan(x) else x,lp.features.toArray()),dtype=np.double )))\
      , numClasses=training.getNumClasses(), categoricalFeaturesInfo={},numTrees=100)
print(modelRandomForest)

# Evaluate the models  

`pixiedust_flightpredict` provides a plugin to the Pixiedust `display` api and adds a menu (look for the plane icon) that computes the accuracy metrics for the models, including the confusion table.

In [ ]:
display(testData)

# Run the predictive model application  

The below cell runs the embedded Pixiedust application. From within the application, you enter your flight information and the models will run and provide the predictions for a delayed flight.

In [ ]:
import pixiedust_flightpredict
from pixiedust_flightpredict import *
pixiedust_flightpredict.flightPredict()

# Get aggregated results for all the flights that have been predicted.
The following cell shows a map with all the airports and flights. Each edge represent an aggregated view of all the flights between 2 airports. Click on it will display a group list of flights that includes how many users are on the same flight

In [ ]:
import pixiedust_flightpredict
pixiedust_flightpredict.displayMapResults()